# Automobile Data Wrangling and Analysis

This notebook performs comprehensive analysis of automobile data including:
- Data loading and cleaning
- Exploratory data analysis
- Statistical analysis and correlations
- Advanced visualizations
- Feature engineering
- Missing data analysis

**Author:** Yash Patil

## 1. Import Libraries and Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import os

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Create organized results directories
base_dir = os.path.dirname(os.getcwd())
results_dir = os.path.join(base_dir, "results")
viz_dir = os.path.join(results_dir, "visualizations")
data_dir = os.path.join(results_dir, "processed_data")
reports_dir = os.path.join(results_dir, "reports")

for directory in [results_dir, viz_dir, data_dir, reports_dir]:
    os.makedirs(directory, exist_ok=True)

print("Setup complete!")
print(f"Results will be saved to: {results_dir}")

## 2. Data Loading

In [ ]:
def load_automobile_data():
    """Load automobile dataset from UCI repository."""
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
    
    # Column names based on UCI dataset documentation
    column_names = [
        "symboling", "normalized-losses", "make", "fuel-type", "aspiration", 
        "num-of-doors", "body-style", "drive-wheels", "engine-location",
        "wheel-base", "length", "width", "height", "curb-weight", 
        "engine-type", "num-of-cylinders", "engine-size", "fuel-system", 
        "bore", "stroke", "compression-ratio", "horsepower", "peak-rpm", 
        "city-mpg", "highway-mpg", "price"
    ]
    
    df = pd.read_csv(url, header=None, names=column_names)
    return df

# Load the data
df = load_automobile_data()

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {len(df.columns)} features")
df.head()

## 3. Data Cleaning and Preprocessing

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

print("Original data info:")
print(f"Shape: {df_clean.shape}")
print(f"Data types: {df_clean.dtypes.value_counts()}")

# Check for missing values represented as '?'
print("\nColumns with '?' values:")
for col in df_clean.columns:
    question_count = (df_clean[col] == '?').sum()
    if question_count > 0:
        print(f"{col}: {question_count} missing values")

In [ ]:
# Replace '?' with NaN
df_clean = df_clean.replace('?', np.nan)

# Convert numeric columns
numeric_columns = [
    'symboling', 'normalized-losses', 'wheel-base', 'length', 'width', 
    'height', 'curb-weight', 'engine-size', 'bore', 'stroke', 
    'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 
    'highway-mpg', 'price'
]

for col in numeric_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Convert fuel efficiency from MPG to L/100km for international standard
df_clean['city-L/100km'] = 235 / df_clean['city-mpg']
df_clean['highway-L/100km'] = 235 / df_clean['highway-mpg']

# Create price categories
df_clean['price_category'] = pd.cut(df_clean['price'], 
                                   bins=[0, 10000, 20000, 50000], 
                                   labels=['Budget', 'Mid-range', 'Luxury'])

print("Data cleaning completed!")
print(f"Final shape: {df_clean.shape}")
print(f"\nMissing values per column:")
missing_summary = df_clean.isnull().sum().sort_values(ascending=False)
print(missing_summary[missing_summary > 0])

## 4. Exploratory Data Analysis

In [ ]:
# Basic statistics
categorical_cols = df_clean.select_dtypes(include=['object']).columns
numerical_cols = df_clean.select_dtypes(include=[np.number]).columns

print(f"Dataset Overview:")
print(f"Shape: {df_clean.shape}")
print(f"Categorical columns: {len(categorical_cols)}")
print(f"Numerical columns: {len(numerical_cols)}")

# Show top categorical summaries
for col in ['make', 'body-style', 'fuel-type']:
    print(f"\n{col.upper()} - Top 5:")
    print(df_clean[col].value_counts().head())

In [ ]:
# Key numerical features summary
key_features = ['price', 'horsepower', 'engine-size', 'city-mpg', 'highway-mpg']
print("Key Numerical Features Summary:")
df_clean[key_features].describe()

## 5. Data Visualizations

In [ ]:
# Basic analysis visualizations
plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
df_clean['price'].dropna().hist(bins=30, edgecolor='black', alpha=0.7)
plt.title('Price Distribution')
plt.xlabel('Price ($)')
plt.ylabel('Frequency')

plt.subplot(2, 3, 2)
make_counts = df_clean['make'].value_counts().head(8)
make_counts.plot(kind='bar')
plt.title('Top 8 Car Makes')
plt.xlabel('Make')
plt.ylabel('Count')
plt.xticks(rotation=45)

plt.subplot(2, 3, 3)
df_clean['body-style'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('Body Style Distribution')

plt.subplot(2, 3, 4)
df_clean.boxplot(column='price', ax=plt.gca())
plt.title('Price Box Plot')
plt.ylabel('Price ($)')

plt.subplot(2, 3, 5)
df_clean['fuel-type'].value_counts().plot(kind='bar', color=['skyblue', 'lightcoral'])
plt.title('Fuel Type Distribution')
plt.xlabel('Fuel Type')
plt.ylabel('Count')

plt.subplot(2, 3, 6)
df_clean.plot.scatter(x='engine-size', y='price', alpha=0.6, ax=plt.gca())
plt.title('Engine Size vs Price')
plt.xlabel('Engine Size')
plt.ylabel('Price ($)')

plt.tight_layout()
plt.savefig(os.path.join(viz_dir, 'basic_analysis.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation analysis with price
price_data = df_clean.dropna(subset=['price'])
correlations = []

for col in numerical_cols:
    if col != 'price' and col in price_data.columns:
        valid_data = price_data[[col, 'price']].dropna()
        if len(valid_data) > 10:
            corr_coef = valid_data[col].corr(valid_data['price'])
            correlations.append({
                'Feature': col,
                'Correlation': corr_coef,
                'Abs_Correlation': abs(corr_coef)
            })

corr_df = pd.DataFrame(correlations).sort_values('Abs_Correlation', ascending=False)

print("Top 10 correlations with price:")
print(corr_df.head(10))

# Visualize top correlations
plt.figure(figsize=(10, 6))
top_corr = corr_df.head(8)
colors = ['red' if x < 0 else 'blue' for x in top_corr['Correlation']]
plt.barh(range(len(top_corr)), top_corr['Correlation'], color=colors, alpha=0.7)
plt.yticks(range(len(top_corr)), top_corr['Feature'])
plt.xlabel('Correlation with Price')
plt.title('Top Feature Correlations with Price')
plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, 'price_correlations.png'), dpi=300, bbox_inches='tight')
plt.show()

## 6. Key Business Insights

In [ ]:
# Generate business insights
insights = []

# Price insights
avg_price = df_clean['price'].mean()
median_price = df_clean['price'].median()
insights.append(f"Average car price: ${avg_price:,.2f}")
insights.append(f"Median car price: ${median_price:,.2f}")

# Top correlation
if len(corr_df) > 0:
    top_corr = corr_df.iloc[0]
    insights.append(f"Strongest price predictor: {top_corr['Feature']} (r = {top_corr['Correlation']:.3f})")

# Market insights
most_common_make = df_clean['make'].mode()[0]
make_count = df_clean['make'].value_counts().iloc[0]
insights.append(f"Most common make: {most_common_make} ({make_count} cars)")

# Performance insights
avg_horsepower = df_clean['horsepower'].mean()
if not np.isnan(avg_horsepower):
    insights.append(f"Average horsepower: {avg_horsepower:.0f} HP")

print("Key Business Insights:")
print("=" * 50)
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

## 7. Data Export

In [ ]:
# Save processed datasets
print("Saving processed data...")

# Save cleaned dataset
df_clean.to_csv(os.path.join(data_dir, 'automobile_data_cleaned.csv'), index=False)

# Save correlation analysis
corr_df.to_csv(os.path.join(reports_dir, 'price_correlations.csv'), index=False)

# Save summary statistics
df_clean.describe().to_csv(os.path.join(reports_dir, 'numeric_summary.csv'))

print("Files saved to organized directories:")
print(f"- Processed data: {data_dir}/")
print(f"- Reports: {reports_dir}/")
print(f"- Visualizations: {viz_dir}/")

## Analysis Complete!

This comprehensive automobile data analysis has provided:

### Data Quality
- Successfully processed 205 automobile records
- Handled missing values and data type conversions
- Created meaningful feature categories

### Key Findings
- Identified strongest predictors of automobile price
- Analyzed market distribution across makes and body styles
- Quantified performance relationships

### Outputs
All results are organized in the `results/` directory:
- **visualizations/**: Charts and plots
- **processed_data/**: Clean datasets
- **reports/**: Analysis summaries